In [ ]:
!nvidia-smi

Sat Mar 30 11:36:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
!pip install emoji
import emoji
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from keras.layers import Embedding
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 3.6 MB/s eta 0:00:00


In [ ]:
# Chargement des données
df = pd.read_csv("/content/drive/MyDrive/nlp_project/reviews_L.csv", encoding='latin-1', header=None)
# Exploration rapide des données
print("Aperçu des données :")
df.head(20)

Aperçu des données :


,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
5,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
6,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
7,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
8,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
9,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   0       1600000 non-null  int64 
 1   1       1600000 non-null  int64 
 2   2       1600000 non-null  object
 3   3       1600000 non-null  object
 4   4       1600000 non-null  object
 5   5       1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


# **Data Preprocessing**

In [ ]:
# fixing the header
df.drop(columns=[3], inplace=True)
df.columns = ['sentiment', 'id', 'date', 'username', 'review']

In [ ]:
# droping unecessary columns
df.drop(columns=['id'], inplace=True)
df.drop(columns=['date'], inplace=True)
df.drop(columns=['username'], inplace=True)

In [ ]:
# Assuming df is your DataFrame and 'sentiment' is the name of the column
df['sentiment'] = df['sentiment'].replace(4, 1)

In [ ]:
df.tail()

,sentiment,review
1599995,1,Just woke up. Having no school is the best fee...
1599996,1,TheWDB.com - Very cool to hear old Walt interv...
1599997,1,Are you ready for your MoJo Makeover? Ask me f...
1599998,1,Happy 38th Birthday to my boo of alll time!!! ...
1599999,1,happy #charitytuesday @theNSPCC @SparksCharity...


Removing duplicates

In [ ]:
# removing duplicates
duplicate_pairs = df[df.duplicated(subset='review', keep=False)]
duplicate_pairs = duplicate_pairs.sort_values(by='review')

In [ ]:
print("Pairs of Duplicate reviews:")
display(duplicate_pairs[['review']])

Pairs of Duplicate reviews:


,review
271472,David must be hospitalized for five days end...
271505,David must be hospitalized for five days end...
1132076,bathroom is clean..... now on to more enjoya...
1132074,bathroom is clean..... now on to more enjoya...
540159,#IMISSCATH #IMISSCATH #IMISSCATH #IMISSCATH #...
...,...
458170,"zofran is not making this work day any easier,..."
464810,~ get SUICIDE !!
463926,~ get SUICIDE !!
1170868,~~~X0X0~~~!!!X0X0!!! I love you!!! &amp;...


In [ ]:
df = df.drop_duplicates(subset='review')

In [ ]:
duplicate_pairs = df[df.duplicated(subset='review', keep=False)]
duplicate_pairs = duplicate_pairs.sort_values(by='review')
print("Pairs of Duplicate reviews:")
display(duplicate_pairs[['review']])

Pairs of Duplicate reviews:


,review


In [ ]:
import re
import pandas as pd
import contractions
# Define the preprocess_text function
def preprocess_text(text):
    # Remove Twitter handles (starting with @)
    text = re.sub(r'@([^\s]+)', '', text)
    # Remove links (starting with http or https)
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    text = re.sub(r'www\.[^\s]+', '', text)
    # Remove ellipses (..)
    text = re.sub(r'\.\.', '', text)
    # Remove ellipses (...)
    text = re.sub(r'\.\.\.', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Lower casing
    text = text.lower()
    # Remove emoji using emoji library
    text = ''.join(c for c in text if not emoji.is_emoji(c))

    # Remove emoticons using r library
    emoticon_patterns = [
        r'[:;][-Oo]?[)d]',     # :) or ;) or :-D or ;-D or :-)
        r'[:;][-Oo]?[(/\\]',   # :( or :/ or :\ or :-/ or :-\
        # r'[:;]p',              # :p or ;p
        # r'[:;][3]',            # :3 or ;3
        # r'=)',                  # =)
        # r'xd',
    ]

    # Remove emoticons using regular expressions
    for pattern in emoticon_patterns:
      text = re.sub(pattern, '', text)

    # Handling abreviation and slang (the most used abreviations in tweets in 2024)
    abbreviation_dict = {
    "lol": "laugh out loud",
    "brb": "be right back",
    "fyi ": "for your information",
    "asap": "as soon as possible",
    "g2g": "got to go",
    "ttyl": "talk to you later",
    "imo" : "in my opinion",
    "fb" : "facebook",
    "msg" : "message",
    "irl" : "in real life",
    "l8r" : "later",
    "thx" : "thanks",
    "ty"  : "thank you",
    "WTF" : "what the freak",
    "2mrw" : "tomorrow",
    "u" : "you",
    "bruh" : "brother",
    "bro" : "brother",
    "ofc" : "of course",
    "amp" : "and"

}
    text = ' '.join(abbreviation_dict.get(word, word) for word in text.split())
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Replace contractions with expanded forms
    text = contractions.fix(text)
    # handling punctuation
    punctuation_pattern = r'[!\"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~¡¿]'
    # Replace the matched punctuation with an empty string
    cleaned_text = re.sub(punctuation_pattern, '', text)
    return text

# Apply preprocessing to each text entry in your DataFrame
df['review'] = df['review'].apply(preprocess_text)

# Display the preprocessed text data
df.head()

,sentiment,review
0,0,awww that is a bummer you shoulda got david c...
1,0,is upset that he cannot update his facebook by...
2,0,i dived many times for the ball managed to sav...
3,0,my whole body feels itchy and like its on fire
4,0,no its not behaving at all i am mad why am i h...


Handling empty rows

In [ ]:
import pandas as pd

def check_for_empty(df, text_column_name):
    rows_without_spaces = []
    for index, row in df.iterrows():
        text = row[text_column_name]
        if not any(c.isalpha() and text[i+1].isalpha() for i, c in enumerate(text[:-1])):
            rows_without_spaces.append(row)
    return pd.DataFrame(rows_without_spaces)

# Assuming df is your existing DataFrame
# Check for rows without spaces between words
empty_rows_df = check_for_empty(df, 'review')

if empty_rows_df.empty:
    print("There are no empty rows.")
else:
    print("Rows without text:")
    print(empty_rows_df)


Rows without text:
         sentiment review
208              0       
249              0       
282              0       
398              0       
430              0       
...            ...    ...
1597684          1       
1598192          1       
1598272          1       
1599494          1       
1599993          1       

[3579 rows x 2 columns]


In [ ]:
import pandas as pd

def drop_empty_rows(df, text_column_name):
    rows_to_drop = []
    for index, row in df.iterrows():
        text = row[text_column_name]
        if not any(c.isalpha() and text[i+1].isalpha() for i, c in enumerate(text[:-1])):
            rows_to_drop.append(index)
    cleaned_df = df.drop(index=rows_to_drop)
    return cleaned_df

# Drop rows without spaces between words
df = drop_empty_rows(df, 'review')

In [ ]:
# Check for rows without spaces between words
empty_rows_df = check_for_empty(df, 'review')

if empty_rows_df.empty:
    print("There are no empty rows.")
else:
    print("Rows without text:")
    print(empty_rows_df)

There are no empty rows.


without tokenazation

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def remove_stopwords(data):
    stop_words = set(stopwords.words('english'))
    words = data.split()  # Split the text into words
    words_filtered = [word for word in words if word.lower() not in stop_words]
    return ' '.join(words_filtered)

In [ ]:
from nltk.corpus import stopwords
df['review']=df['review'].apply(lambda z: remove_stopwords(z))

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
#Lemmatize the dataset
from nltk.stem import WordNetLemmatizer


def lemma_traincorpus(data):
    lemmatizer=WordNetLemmatizer()
    out_data=""
    for words in data:
        out_data+= lemmatizer.lemmatize(words)
    return out_data


df['review'] = df['review'].apply(lambda x: lemma_traincorpus(x))

In [ ]:
df_for_LLM=df.copy()

# **LLM**

In [ ]:
!pip install -qU \
     accelerate \
     peft \
     einops \
     datasets \
     bitsandbytes \
     trl \
     tramsformers \
     datasets
!pip install bitsandbytes
!pip install datasets
!pip install peft
!pip install trl

**BERT**

In [ ]:
df_for_LLM.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1577887 entries, 0 to 1599999
Data columns (total 2 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentiment  1577887 non-null  int64 
 1   review     1577887 non-null  object
dtypes: int64(1), object(1)
memory usage: 36.1+ MB


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2).cuda()
#  df_for_LLM is the DataFrame containing text and labels
texts = df_for_LLM['review'].tolist()
labels = df_for_LLM['sentiment'].tolist()

# Sample 10% from each label category
# You may need to adjust this sampling process based on your specific label distribution
sampled_texts = []
sampled_labels = []
for label in set(labels):
    label_indices = [i for i, l in enumerate(labels) if l == label]
    sampled_indices = np.random.choice(label_indices, size=int(len(label_indices)*0.01), replace=False)
    sampled_texts.extend([texts[i] for i in sampled_indices])
    sampled_labels.extend([labels[i] for i in sampled_indices])

# Split data into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(sampled_texts, sampled_labels, test_size=0.2, random_state=42)

# Prepare data
train_inputs = tokenizer(train_texts, padding=True, truncation=True, return_tensors='pt')  # texts is your list of input text
train_labels = torch.tensor(train_labels).cuda()  # labels is your list of binary sentiment labels
test_inputs = tokenizer(test_texts, padding=True, truncation=True, return_tensors='pt')
test_labels = torch.tensor(test_labels).cuda()

# Create DataLoaders for train and test sets
train_dataset = TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = TensorDataset(test_inputs['input_ids'], test_inputs['attention_mask'], test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=32)

# Fine-tuning parameters
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 5

# Fine-tuning loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids.cuda(), attention_mask=attention_mask.cuda(), labels=labels.cuda())
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1}: Average Loss: {total_loss / len(train_dataloader)}")

# Evaluation
model.eval()
predictions = []
true_labels = []
for batch in test_dataloader:
    input_ids, attention_mask, labels = batch
    with torch.no_grad():
        outputs = model(input_ids=input_ids.cuda(), attention_mask=attention_mask.cuda())
        logits = outputs.logits
        predicted_labels = torch.argmax(logits, dim=1)
        predictions.extend(predicted_labels.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
print("Test Accuracy:", accuracy)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1: Average Loss: 0.4658259794567571
Epoch 2: Average Loss: 0.2996064369011708
Epoch 3: Average Loss: 0.15603319877720992
Epoch 4: Average Loss: 0.08355874381077504
Epoch 5: Average Loss: 0.06363286516018272
Test Accuracy: 0.799873537780588


In [ ]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(true_labels, predictions)
print("Classification Report:")
print(report)


Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.80      0.80      1563
           1       0.81      0.80      0.80      1600

    accuracy                           0.80      3163
   macro avg       0.80      0.80      0.80      3163
weighted avg       0.80      0.80      0.80      3163



**GPT2**

In [ ]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, GPT2Config
import torch
from transformers import AdamW
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np

# Load pre-trained GPT-2 model and tokenizer
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({'pad_token': ''})  # Add token as the padding token

config = GPT2Config.from_pretrained(model_name)
config.pad_token_id = tokenizer.pad_token_id

model = GPT2ForSequenceClassification.from_pretrained(model_name, config=config).cuda()  # Assuming binary classification

# Assuming df_for_LLM is your DataFrame containing text and labels
texts = df_for_LLM['review'].tolist()  # Assuming the column containing text is named 'review'
labels = df_for_LLM['sentiment'].tolist()  # Assuming the column containing labels is named 'sentiment'

# Sample 10% from each label category
# You may need to adjust this sampling process based on your specific label distribution
sampled_texts = []
sampled_labels = []
for label in set(labels):
    label_indices = [i for i, l in enumerate(labels) if l == label]
    sampled_indices = np.random.choice(label_indices, size=int(len(label_indices)*0.01), replace=False)
    sampled_texts.extend([texts[i] for i in sampled_indices])
    sampled_labels.extend([labels[i] for i in sampled_indices])

# Check if sampled_texts and sampled_labels are not empty
if not sampled_texts or not sampled_labels:
    raise ValueError("Sampled data is empty. Please check your sampling process.")

# Split data into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(sampled_texts, sampled_labels, test_size=0.2, random_state=42)

# Check if train_texts and test_texts are not empty
if not train_texts or not test_texts:
    raise ValueError("Train or test data is empty. Please check your sampling process.")
# Fine-tuning loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for text, label in zip(train_texts, train_labels):
        if text.strip() == "":
            continue  # Skip empty text
        inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(model.device)
        label_tensor = torch.tensor(label).unsqueeze(0).to(model.device)  # Convert label to tensor

        optimizer.zero_grad()
        outputs = model(**inputs, labels=label_tensor)  # Pass label_tensor to labels
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1}: Average Loss: {total_loss / len(train_texts)}")

# Evaluation
model.eval()
predictions = []
true_labels = []
for text, label in zip(test_texts, test_labels):
    if text.strip() == "":
        continue  # Skip empty text
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(model.device)
    label_tensor = torch.tensor(label).unsqueeze(0).to(model.device)  # Convert label to tensor

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=-1).item()
        predictions.append(predicted_label)
        true_labels.append(label)

accuracy = accuracy_score(true_labels, predictions)
print("Test Accuracy:", accuracy)



Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1: Average Loss: 2.0204848091738064
Epoch 2: Average Loss: 2.0437264248464566
Epoch 3: Average Loss: 2.056541714335541
Epoch 4: Average Loss: 2.061774228178358
Epoch 5: Average Loss: 2.041830820711786
Epoch 6: Average Loss: 2.054973945307012
Epoch 7: Average Loss: 2.0350046381552542
Epoch 8: Average Loss: 2.0469528770474064
Epoch 9: Average Loss: 2.0517041105734948
Epoch 10: Average Loss: 2.0481950210645037
Test Accuracy: 0.5011107584893685


**PHI2**

In [ ]:
# Assuming 'sentiment' is the column name in your DataFrame
df_sentiment_0 = df_for_LLM[df_for_LLM['sentiment'] == 0].sample(n=5000, random_state=42)
df_sentiment_1 = df_for_LLM[df_for_LLM['sentiment'] == 1].sample(n=5000, random_state=42)

# Assuming df_sentiment_0 and df_sentiment_1 are already defined
df_for_LLM = pd.concat([df_sentiment_0, df_sentiment_1], ignore_index=True)

In [ ]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments,
    pipeline, logging
)
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
df_for_LLM

,sentiment,review
0,0,my ipod can stop this restore to factory setti...
1,0,headache waaaaaaaa
2,0,i am going to bed and sleeping in tomorrow mor...
3,0,its the same old song but with a different mea...
4,0,i did not finish watching it the website kept ...
...,...,...
9995,1,welcome to bornrecyclingcouk nathanbonnalataya...
9996,1,looooong day yesterday getting to hotel for ba...
9997,1,had the penguin tonightbut stuck w a black bea...
9998,1,ghostbusters the game comes out today and i fo...


In [ ]:
# Replace 0 with 'negative' and 1 with 'positive' in the 'sentiment' column
df_for_LLM['sentiment'] = df_for_LLM['sentiment'].replace({0: 'negative', 1: 'positive'})

In [ ]:
df_for_LLM

,sentiment,review
0,negative,my ipod can stop this restore to factory setti...
1,negative,headache waaaaaaaa
2,negative,i am going to bed and sleeping in tomorrow mor...
3,negative,its the same old song but with a different mea...
4,negative,i did not finish watching it the website kept ...
...,...,...
9995,positive,welcome to bornrecyclingcouk nathanbonnalataya...
9996,positive,looooong day yesterday getting to hotel for ba...
9997,positive,had the penguin tonightbut stuck w a black bea...
9998,positive,ghostbusters the game comes out today and i fo...


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from datasets import Dataset

In [ ]:
# Split the dataset into training and testing sets for each sentiment
X_train = list()
X_test = list()

for sentiment in ["positive", "negative"]:
    train, test = train_test_split(df_for_LLM[df_for_LLM.sentiment==sentiment],
                                   train_size=500,
                                   test_size=500,
                                   random_state=42)
    X_train.append(train)
    X_test.append(test)

In [ ]:
# Concatenate the training and testing sets for each sentiment
X_train = pd.concat(X_train).sample(frac=1, random_state=10)
X_test = pd.concat(X_test)

In [ ]:
# Create an evaluation set from the remaining data points
eval_idx = [idx for idx in df_for_LLM.index if idx not in list(train.index) + list(test.index)]
X_eval = df_for_LLM[df_for_LLM.index.isin(eval_idx)]
X_eval = (X_eval
          .groupby('sentiment', group_keys=False)
          .apply(lambda x: x.sample(n=50, random_state=10, replace=True)))
X_train = X_train.reset_index(drop=True)

In [ ]:
# Function to generate a prompt for training data
def generate_prompt(data_point):
    return f"""The sentiment of the following phrase: '{data_point["review"]}' is
            \n\n Positive
            \n Negative
            \n\nSolution: The correct option is {data_point["sentiment"]}""".strip()

# Function to generate a prompt for evaluation data
def generate_test_prompt(data_point):
    return f"""The sentiment of the following phrase: '{data_point["review"]}' is
            \n\n Positive
            \n Negative
            \n\nSolution: The correct option is""".strip()

In [ ]:
# Create DataFrames for training and evaluation data with generated prompts
X_train = pd.DataFrame(X_train.apply(generate_prompt, axis=1),
                       columns=["review"])
X_eval = pd.DataFrame(X_eval.apply(generate_prompt, axis=1),
                      columns=["review"])

In [ ]:
X_train

,text
0,The sentiment of the following phrase: 'caused...
1,The sentiment of the following phrase: 'nearly...
2,The sentiment of the following phrase: 'would ...
3,The sentiment of the following phrase: 'bought...
4,The sentiment of the following phrase: 'great ...
...,...
995,The sentiment of the following phrase: 'yes se...
996,The sentiment of the following phrase: 'living...
997,The sentiment of the following phrase: 'watche...
998,The sentiment of the following phrase: 'christ...


In [ ]:
# Extract true labels for testing set
y_true = X_test.sentiment

# Create a DataFrame for testing data with generated prompts
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["review"])

In [ ]:
# Create datasets for training and evaluation using the datasets library
train_data = Dataset.from_pandas(X_train)
eval_data = Dataset.from_pandas(X_eval)

In [ ]:
def evaluate(y_true, y_pred):
    # Define labels and mapping for sentiment categories
    labels = ['positive', 'negative']
    mapping = {'positive': 1, 'negative': 0}

    # Function to map labels to numerical values
    def map_func(x):
        return mapping.get(x, 1)

    # Map true and predicted labels using the mapping function
    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)

    # Calculate overall accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Overall Accuracy: {accuracy:.3f}')

    # Generate accuracy report for each sentiment label
    unique_labels = set(y_true)  # Get unique labels

    for label in unique_labels:
        # Find indices where true labels match the current sentiment label
        label_indices = [i for i in range(len(y_true)) if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]

        # Calculate accuracy for the current sentiment label
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {labels[label]}: {accuracy:.3f}')

    # Generate classification report for precision, recall, and F1-score
    class_report = classification_report(y_true=y_true, y_pred=y_pred, target_names=labels)
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix to analyze the performance of the model
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1, 2])
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Configuration for quantization
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

# Load pre-trained model with quantization config
model_name = "microsoft/phi-2"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=bnb_config,
)

# Customize model configuration
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load tokenizer for the same pre-trained model
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
)

# Set pad_token to eos_token for consistency
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from tqdm import tqdm
from transformers import pipeline

def predict(X_test, model, tokenizer):
    # List to store predicted labels
    y_pred = []

    # Loop through each data point in the test set
    for i in tqdm(range(len(X_test))):
        # Get the prompt from the test set
        prompt = X_test.iloc[i]["review"]

        # Create a text generation pipeline using the specified model and tokenizer
        pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens=3,
                        temperature=0.0,
                       )

        # Generate text based on the prompt
        result = pipe(prompt, pad_token_id=pipe.tokenizer.eos_token_id)

        # Extract the generated text and identify the sentiment label
        answer = result[0]['generated_text'].split("The correct option is")[-1].lower()
        if "positive" in answer:
            y_pred.append("positive")
        elif "negative" in answer:
            y_pred.append("negative")
        elif "neutral" in answer:
            y_pred.append("neutral")
        else:
            y_pred.append("none")

    return y_pred

In [ ]:
y_pred_phi2 = predict(X_test, model, tokenizer)

  0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
100%|██████████| 1000/1000 [03:59<00:00,  4.17it/s]


In [ ]:
evaluate(y_true, y_pred_phi2)

Overall Accuracy: 0.591
Accuracy for label positive: 0.372
Accuracy for label negative: 0.810

Classification Report:
              precision    recall  f1-score   support

    positive       0.66      0.37      0.48       500
    negative       0.56      0.81      0.66       500

    accuracy                           0.59      1000
   macro avg       0.61      0.59      0.57      1000
weighted avg       0.61      0.59      0.57      1000


Confusion Matrix:
[[186 314   0]
 [ 95 405   0]
 [  0   0   0]]


In [ ]:
import plotly.graph_objects as go

# Metrics values
accuracy = 0.349
precision_positive = 0.92
precision_negative = 0.39

recall_positive = 0.07

recall_negative = 0.11

f1_positive = 0.14
f1_negative = 0.17

# Labels
labels = ['Accuracy', 'Precision', 'Recall', 'F1-Score']

# Values for each sentiment label
positive_values = [accuracy, precision_positive, recall_positive, f1_positive]

negative_values = [accuracy, precision_negative, recall_negative, f1_negative]

# Grouped Bar Chart
fig = go.Figure()

fig.add_trace(go.Bar(
    x=labels,
    y=positive_values,
    name='Positive',
    marker_color='#FF69B4'
))



fig.add_trace(go.Bar(
    x=labels,
    y=negative_values,
    name='Negative',
    marker_color='#9370DB'
))

fig.update_layout(
    barmode='group',
    title='Metrics Overview by Sentiment',
    xaxis=dict(title='Metrics'),
    yaxis=dict(title='Values'),
    legend=dict(title='Sentiment'),
)

fig.show()

In [ ]:
#Finetuning

In [ ]:
import re
import torch
#  Get the Maximum Number of Layers in the Model.
def get_num_layers(model):
    numbers = set()
    for name, _ in model.named_parameters():
        for number in re.findall(r'\d+', name):
            numbers.add(int(number))
    return max(numbers)
#  Get Names of Last Layer Linear Modules.
def get_last_layer_linears(model):
    names = []

    num_layers = get_num_layers(model)
    for name, module in model.named_modules():
        if str(num_layers) in name and not "encoder" in name:
            if isinstance(module, torch.nn.Linear):
                names.append(name)
    return names

In [ ]:
# Configuration for LORA model
peft_config =LoraConfig(
    r=16,  # dimension of the updated matrices
    lora_alpha=64,  # parameter for scaling
    target_modules=[
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj"],
    lora_dropout=0.1,  # dropout probability for layers
    bias="none",
    task_type="CAUSAL_LM",
)

# Training arguments
training_arguments = TrainingArguments(
    output_dir="logs",
    num_train_epochs=10,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,  # 4
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    evaluation_strategy="epoch"
)

# Trainer setup
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="review",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
    max_seq_length=512,
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning:

Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)



In [ ]:
# Train model
trainer.train()
# Save trained model
trainer.model.save_pretrained("trained-model")

Epoch,Training Loss,Validation Loss
1,1.342200,1.822102
2,1.353500,1.692491
3,1.324700,1.672213
4,1.217400,1.687727
5,1.178900,1.709926
6,1.014800,1.758076
7,0.976400,1.782878
8,0.847500,1.829077
9,0.815700,1.839417
10,0.891300,1.844622


In [ ]:
y_pred_finetuned_phi2 = predict(X_test, model, tokenizer)
evaluate(y_true, y_pred_finetuned_phi2)

  0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning:

`do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.

100%|██████████| 1000/1000 [05:35<00:00,  2.98it/s]

Overall Accuracy: 0.819
Accuracy for label positive: 0.826
Accuracy for label negative: 0.812

Classification Report:
              precision    recall  f1-score   support

    positive       0.81      0.83      0.82       500
    negative       0.82      0.81      0.82       500

    accuracy                           0.82      1000
   macro avg       0.82      0.82      0.82      1000
weighted avg       0.82      0.82      0.82      1000


Confusion Matrix:
[[413  87   0]
 [ 94 406   0]
 [  0   0   0]]
